In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Deployment and Integration Testing\n",
    "\n",
    "This notebook tests the complete system integration and deployment readiness:\n",
    "- API endpoint testing\n",
    "- Performance benchmarking\n",
    "- MongoDB integration\n",
    "- Pipeline orchestration\n",
    "\n",
    "## Setup"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "import sys\n",
    "sys.path.append('..')\n",
    "\n",
    "import time\n",
    "import requests\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "from pymongo import MongoClient\n",
    "\n",
    "from src.database.mongo.connection import get_database\n",
    "from src.database.mongo.queries import insert_attendance_record, get_employee_records\n",
    "from src.pipeline.model_pipeline import ModelPipeline\n",
    "from src.pipeline.result_aggregator import ResultAggregator\n",
    "from src.utils.logging import setup_logger\n",
    "\n",
    "# Setup logging\n",
    "logger = setup_logger('deployment_testing')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. API Endpoint Testing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def test_api_endpoints(base_url='http://localhost:8000'):\n",
    "    endpoints = {\n",
    "        'health': '/health',\n",
    "        'predict': '/api/v1/predict',\n",
    "        'employees': '/api/v1/employees',\n",
    "        'attendance': '/api/v1/attendance'\n",
    "    }\n",
    "    \n",
    "    results = []\n",
    "    for name, endpoint in endpoints.items():\n",
    "        try:\n",
    "            start_time = time.time()\n",
    "            response = requests.get(f'{base_url}{endpoint}')\n",
    "            latency = time.time() - start_time\n",
    "            \n",
    "            results.append({\n",
    "                'endpoint': name,\n",
    "                'status_code': response.status_code,\n",
    "                'latency': latency,\n",
    "                'success': response.status_code == 200\n",
    "            })\n",
    "        except Exception as e:\n",
    "            logger.error(f'Error testing endpoint {name}: {str(e)}')\n",
    "            results.append({\n",
    "                'endpoint': name,\n",
    "                'status_code': None,\n",
    "                'latency': None,\n",
    "                'success': False,\n",
    "                'error': str(e)\n",
    "            })\n",
    "    \n",
    "    return pd.DataFrame(results)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Performance Benchmarking"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def benchmark_pipeline(test_images, n_iterations=10):\n",
    "    pipeline = ModelPipeline()\n",
    "    metrics = []\n",
    "    \n",
    "    for img_path in test_images:\n",
    "        img_metrics = []\n",
    "        for _ in range(n_iterations):\n",
    "            start_time = time.time()\n",
    "            \n",
    "            # Measure each step\n",
    "            t0 = time.time()\n",
    "            faces = pipeline.detect_faces(img_path)\n",
    "            t1 = time.time()\n",
    "            \n",
    "            features = pipeline.extract_features(faces)\n",
    "            t2 = time.time()\n",
    "            \n",
    "            predictions = pipeline.predict(features)\n",
    "            t3 = time.time()\n",
    "            \n",
    "            img_metrics.append({\n",
    "                'image': img_path,\n",
    "                'detection_time': t1 - t0,\n",
    "                'feature_extraction_time': t2 - t1,\n",
    "                'prediction_time': t3 - t2,\n",
    "                'total_time': t3 - t0,\n",
    "                'faces_detected': len(faces)\n",
    "            })\n",
    "        \n",
    "        metrics.extend(img_metrics)\n",
    "    \n",
    "    return pd.DataFrame(metrics)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. MongoDB Integration Testing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def test_database_integration():\n",
    "    try:\n",
    "        # Test connection\n",
    "        db = get_database()\n",
    "        \n",
    "        # Test write operations\n",
    "        test_record = {\n",
    "            'employee_id': 'test_employee',\n",
    "            'timestamp': time.time(),\n",
    "            'status': 'present'\n",
    "        }\n",
    "        insert_result = insert_attendance_record(test_record)\n",
    "        \n",
    "        # Test read operations\n",
    "        records = get_employee_records('test_employee')\n",
    "        \n",
    "        return {\n",
    "            'connection_success': True,\n",
    "            'write_success': insert_result is not None,\n",
    "            'read_success': len(records) > 0\n",
    "        }\n",
    "    except Exception as e:\n",
    "        logger.error(f'Database integration test failed: {str(e)}')\n",
    "        return {\n",
    "            'connection_success': False,\n",
    "            'write_success': False,\n",
    "            'read_success': False,\n",
    "            'error': str(e)\n",
    "        }"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. System Load Testing"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def load_test_api(base_url, n_requests=100, concurrent=10):\n",
    "    import concurrent.futures\n",
    "    \n",
    "    def make_request():\n",
    "        try:\n",
    "            start_time = time.time()\n",
    "            response = requests.post(\n",
    "                f'{base_url}/api/v1/predict',\n",
    "                files={'image': open('path/to/test/image.jpg', 'rb')}\n",
    "            )\n",
    "            return {\n",
    "                'success': response.status_code == 200,\n",
    "                'latency': time.time() - start_time,\n",
    "                'status_code': response.status_code\n",
    "            }\n",
    "        except Exception as e:\n",
    "            return {\n",
    "                'success': False,\n",
    "                'latency': None,\n",
    "                'error': str(e)\n",
    "            }\n",
    "    \n",
    "    results = []\n",
    "    with concurrent.futures.ThreadPoolExecutor(max_workers=concurrent) as executor:\n",
    "        futures = [executor.submit(make_request) for _ in range(n_requests)]\n",
    "        for future in concurrent.futures.as_completed(futures):\n",
    "            results.append(future.result())\n",
    "    \n",
    "    return pd.DataFrame(results)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. End-to-End Pipeline Test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "source": [
    "def test_end_to_end_pipeline(test_image_path):\n",
    "    pipeline = ModelPipeline()\n",
    "    aggregator = ResultAggregator()\n",
    "    \n    try:\n",
    "        # Process image through pipeline\n",
    "        faces = pipeline.detect_faces(test_image_path)\n",
    "        features = pipeline.extract_features(faces)\n",
    "        predictions = pipeline.predict(features)\n",
    "        \n",
    "        # Aggregate results\n",
    "        final_results = aggregator.combine_results(predictions)\n",
    "        \n",
    "        # Store results in database\n",
    "        db = get_database()\n",
    "        for result in final_results:\n",
    "            insert_attendance_record({\n",
    "                'employee_id': result['employee_id'],\n",
    "                'timestamp': time.time(),\n",
    "                'confidence': result['confidence'],\n",
    "                'additional_metrics': result['metrics']\n",
    "            })\n",
    "        \n",
    "        return {\n",
    "            'success': True,\n",
    "            'faces_detected': len(faces),\n",
    "            'predictions_made': len(predictions),\n",
    "            'results_stored': len(final_results)\n",
    "        }\n",
    "    \n",
    "    except Exception as e:\n",
    "        logger.error(f'End-to-end pipeline test failed: {str(e)}')\n",
    "        return {\n",
    "            'success': False,\n",
    "            'error': str(e)\n",
    "        }"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}